# Import Library

In [1]:
import keras
import numpy as np
import pandas as pd
import zipfile
import os
import shutil
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.densenet import DenseNet201
from keras.preprocessing import image
from keras import models, layers, optimizers

# Unzip file

In [2]:
with zipfile.ZipFile("train.zip", "r") as zip_ref:
  zip_ref.extractall("./")

with zipfile.ZipFile("test1.zip", "r") as zip_ref:
  zip_ref.extractall("./")

FileNotFoundError: ignored

# Print total train images and test images

In [ ]:
print("Total train images:")
print(len([f for f in os.listdir('./train') if os.path.isfile(os.path.join('./train',f))]))
print("Total test images:")
print(len([f for f in os.listdir('./test1') if os.path.isfile(os.path.join('./test1',f))]))

# Print Image

In [ ]:
for dirname, _, filenames in os.walk('./train'):
  for filename in filenames:
    img = plt.imread(os.path.join(dirname, filename))
    plt.figure(0)
    plt.imshow(img)
    break

# Creating folder for training, testing and validating

In [ ]:
base_dir = './'

train_dir = os.path.join(base_dir, 'train_dir')
os.mkdir(train_dir)
val_dir = os.path.join(base_dir, 'val_dir')
os.mkdir(val_dir)
test_dir = os.path.join(base_dir, 'test_dir')
os.mkdir(test_dir)

train_cats = os.path.join(train_dir, 'cats')
os.mkdir(train_cats)
train_dogs = os.path.join(train_dir, 'dogs')
os.mkdir(train_dogs)

val_cats = os.path.join(val_dir, 'cats')
os.mkdir(val_cats)
val_dogs = os.path.join(val_dir, 'dogs')
os.mkdir(val_dogs)

test_cats = os.path.join(test_dir, 'cats')
os.mkdir(test_cats)
test_dogs = os.path.join(test_dir, 'dogs')
os.mkdir(test_dogs)

# Copy Images of train, test and validating (80% for train, 20% test and 10% for validating)

In [ ]:
files = ['cat.{}.jpg'.format(i) for i in range(8750)]
for filename in files:
    src = os.path.join('./train', filename)
    dst = os.path.join(train_cats, filename)
    shutil.copyfile(src, dst)
    
files = ['cat.{}.jpg'.format(i) for i in range(8750, 10000)]
for filename in files:
    src = os.path.join('./train', filename)
    dst = os.path.join(val_cats, filename)
    shutil.copyfile(src, dst)
    
files = ['cat.{}.jpg'.format(i) for i in range(10000, 12500)]
for filename in files:
    src = os.path.join('./train', filename)
    dst = os.path.join(test_cats, filename)
    shutil.copyfile(src, dst)
    
files = ['dog.{}.jpg'.format(i) for i in range(8750)]
for filename in files:
    src = os.path.join('./train', filename)
    dst = os.path.join(train_dogs, filename)
    shutil.copyfile(src, dst)

files = ['dog.{}.jpg'.format(i) for i in range(8750, 10000)]
for filename in files:
    src = os.path.join('./train', filename)
    dst = os.path.join(val_dogs, filename)
    shutil.copyfile(src, dst)
    
files = ['dog.{}.jpg'.format(i) for i in range(10000, 12500)]
for filename in files:
    src = os.path.join('./train', filename)
    dst = os.path.join(test_dogs, filename)
    shutil.copyfile(src, dst)

In [ ]:
#i use 10000 dataset
print('Train cats: ', len(os.listdir(train_cats)))
print('Train dogs: ', len(os.listdir(train_dogs)))


# Preprocessing Data

In [ ]:
data_scale = ImageDataGenerator(rescale = 1./255)
img_size = (150,150)
batch = 200

train = data_scale.flow_from_directory(train_dir, target_size=img_size, batch_size = batch, class_mode = 'binary')

val_generator = data_scale.flow_from_directory(val_dir, target_size = img_size, batch_size = batch, class_mode = 'binary')

In [ ]:
conv = DenseNet201(weights='imagenet', include_top = False, input_shape = (150,150,3))

conv.trainable = False

model = models.Sequential()
model.add(conv)
model.add(layers.Flatten())
model.add(layers.Dense(250, activation='relu'))
model.add(layers.Dense(1, activation = 'sigmoid'))

model.compile(loss='binary_crossentropy', optimizer = optimizers.RMSprop(lr = 0.01), metrics=['acc'])

In [ ]:
callbacks_list = [
    keras.callbacks.EarlyStopping(
    monitor='acc',
    patience=1),
    keras.callbacks.ModelCheckpoint(
    filepath='vgg16_tuned.h5',
    monitor='loss',
    svae_best_only=True)
]

In [ ]:
training = model.fit(train, validation_data = val_generator, steps_per_epoch = train.n//batch, validation_steps = val_generator.n//batch, epochs = 2, callbacks = callbacks_list)

# Print Accuracy and Loss 

In [ ]:
print('Epoch 1: ', training.history['acc'][0], 'Loss: ', training.history['loss'][0])
print('Epoch 2: ', training.history['acc'][1], 'Loss: ', training.history['loss'][1])

# Fine Tuning

In [ ]:
conv.trainable = False

for layer in conv.layers[:-10]:
    layer.trainable = False

In [ ]:
model.compile(loss= 'binary_crossentropy', optimizer=optimizers.RMSprop(lr = 0.01), metrics=['acc'])

training = model.fit(train, validation_data=val_generator, steps_per_epoch=train.n//batch, validation_steps=val_generator.n//batch, epochs = 2, callbacks= callbacks_list)

In [ ]:
print('Epoch 1: ', training.history['acc'][0], 'Loss: ', training.history['loss'][0])
print('Epoch 2: ', training.history['acc'][1], 'Loss: ', training.history['loss'][1])